<a href="https://colab.research.google.com/github/ShivaShirsath/Plant-Disease-Diagnosis-Flask/blob/master/pddf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install flask-ngrok pyngrok gevent

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 18.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 105.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 28.5 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=324e76c06ca4f4b44662607d3bd815f994c09237ae8545e63f65eee342c1a3bc
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [6]:
! ngrok authtoken 2N6WNOvhe2XPK2SkHwaz7gwn0k0_21vYzWJsMBdgRaU7wYrkG

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [3]:
%cd /content
! git clone https://github.com/sudarshanbhavar/Plant-Disease-Diagnosis-Flask.git
%cd /content/Plant-Disease-Diagnosis-Flask

/content
Cloning into 'Plant-Disease-Diagnosis-Flask'...
remote: Enumerating objects: 84, done.
remote: Counting objects: 100% (37/37), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 84 (delta 18), reused 17 (delta 6), pack-reused 47
Unpacking objects: 100% (84/84), 77.52 MiB | 8.57 MiB/s, done.
/content/Plant-Disease-Diagnosis-Flask


In [ ]:
import os
import tensorflow as tf
import numpy as np
from tensorflow import keras
from skimage import io
from tensorflow.keras.preprocessing import image

# Flask utils
from flask import Flask, redirect, url_for, request, render_template
from werkzeug.utils import secure_filename
from gevent.pywsgi import WSGIServer

# Define a flask app
app = Flask(__name__)

from flask_ngrok import run_with_ngrok

run_with_ngrok(app)

model =tf.keras.models.load_model('PlantDNet.h5',compile=False)

def model_predict(img_path, model):
    img = image.load_img(img_path, grayscale=False, target_size=(64, 64))
    show_img = image.load_img(img_path, grayscale=False, target_size=(64, 64))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = np.array(x, 'float32')
    x /= 255
    preds = model.predict(x)
    return preds


@app.route('/', methods=['GET'])
def index():
    # Main page
    return render_template('index.html')


@app.route('/predict', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        # Get the file from post request
        f = request.files['file']

        # Save the file to ./uploads
        basepath = os.getcwd()
        file_path = os.path.join(
            basepath, 'uploads', secure_filename(f.filename))
        f.save(file_path)

        # Make prediction
        preds = model_predict(file_path, model)
        print(preds[0])

       # x = x.reshape([64, 64]);
        disease_class = ['Pepper__bell___Bacterial_spot', 'Pepper__bell___healthy', 'Potato___Early_blight',
                         'Potato___Late_blight', 'Potato___healthy', 'Tomato_Bacterial_spot', 'Tomato_Early_blight',
                         'Tomato_Late_blight', 'Tomato_Leaf_Mold', 'Tomato_Septoria_leaf_spot',
                         'Tomato_Spider_mites_Two_spotted_spider_mite', 'Tomato__Target_Spot',
                         'Tomato__Tomato_YellowLeaf__Curl_Virus', 'Tomato__Tomato_mosaic_virus', 'Tomato_healthy']
        disease_classs = ['Pomegranate___Bacterial_spot', 'Pomegranate___healthy', 'Pomegranate___Early_blight',
                         'Pomegranate___Late_blight', 'Pomegranate___healthy', 'Pomegranate_Bacterial_spot', 'Pomegranate_Early_blight',
                         'Pomegranate_Late_blight', 'Pomegranate_Leaf_Mold', 'Pomegranate_Septoria_leaf_spot',
                         'Pomegranate_Spider_mites_Two_spotted_spider_mite', 'Pomegranate__Target_Spot',
                         'Pomegranate_YellowLeaf__Curl_Virus', 'Pomegranate_mosaic_virus', 'Pomegranate_healthy']
        medicines = [
                        'copper fungicide',
                        '',
                        'copper fungicide',
                        'copper fungicide',
                        '',
                        'copper fungicide',
                        'copper fungicide',
                        'copper fungicide',
                        'copper fungicide',
                        'insecticide',
                        'phytoalexin inducer',
                        'virus free seeds',
                        ''
                        ]

        a = preds[0]
        ind=np.argmax(a)
        print('Prediction:', disease_classs[ind])
        result=disease_classs[ind] + ", Medicine :"+ medicines[ind]
        return result
    return None


if __name__ == '__main__':
    # app.run(port=5002, debug=True)

    # Serve the app with gevent
    #http_server = WSGIServer(('', 5000), app)
    #http_server.serve_forever()
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://f0aa-34-168-67-63.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [29/May/2023 15:26:37] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 15:26:38] "GET /static/css/test.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 15:26:38] "GET /static/css/bootstrap.min.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 15:26:38] "GET /static/js/jquery.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 15:26:38] "GET /static/js/bootstrap.min.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 15:26:38] "GET /static/js/newjs.js HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 15:26:39] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/May/2023 15:26:40] "GET /favicon.ico HTTP/1.1" 404 -


1/1 [==============================] - 2s 2s/step


INFO:werkzeug:127.0.0.1 - - [29/May/2023 15:26:58] "POST /predict HTTP/1.1" 200 -


[1.2597222e-03 9.8835957e-01 2.2875539e-03 1.2453772e-07 8.2023020e-05
 2.3833149e-08 6.3079945e-03 3.8441583e-05 4.2140271e-04 1.2234064e-03
 3.7283147e-08 4.5516849e-08 1.9596635e-06 1.1841242e-05 5.8394039e-06]
Prediction: Pomegranate___healthy
